# 迷宫搜索

## 1.导入所需模块

In [ ]:
from nxbot import Robot,event,bgr8_to_jpeg
import torchvision
import torch
import torchvision.transforms as transforms
import torch.nn.functional as F
import cv2
import PIL
import numpy as np
from IPython.display import display
import ipywidgets
import traitlets
import ipywidgets.widgets as widgets
import time

## 2.加载已训练好的模型

* torch模型加载方式：

1. torch.load('model'),加载模型与网络结构。
2. model.load_state_dict(torch.load('model')),只加载模型参数， 因此在加载参数之前需要先加载网络结构。

> 使用哪一种方法取决于在训练时模型是已哪一种方式保存的，如果保存模型是通过“torch.save(model, 'model_path')”，加载模型就使用第一种方法;

> 如果保存模型是通过“torch.save(model.state_dict(), 'model_path')”那么加载模型时就使用第二种方法。


* 在这个实验中我们会同时使用两个模型，一个是建筑物识别模型，一个是路径跟踪模型，因此我们需要把两个模型都加载进来。

1. 路径跟踪模型；
> 加载“resnet18”模型；如果我们保存的模型中包含有网络结构和模型参数，就可以直接加载该模型；使用GPU来进行计算。
2. 标识牌识别模型。
> 加载“alexnet”模型；如果我们只保存了模型的参数，那么就需要先加载“alexnet”网络结构，然后改变网络的输出节点，这里为7，因为我们训练的时候有7个类别；然后通过该网络结构再加载模型参数；使用GPU来进行计算。

In [ ]:
# 自己训练的模型路径为 'students_models/steering.pth'
road_model= torch.load('../../models/maze_models/road_following_model/origin_model/steering.pth')
road_model = road_model.cuda()
# .half()可以降低部分精度从而提高识别速度
road_model = road_model.half().eval()

detect_model = torchvision.models.alexnet(pretrained=False)
detect_model.classifier[6] = torch.nn.Linear(detect_model.classifier[6].in_features, 7)
# 自己训练的模型路径为 'students_models/signal.pth'
detect_model.load_state_dict(torch.load('../../models/maze_models/signal_model/origin_model/signal.pth'))
# 这里alexnet精度不是很高不建议用 half()
detect_model.cuda().eval()

## 3.创建可视化小部件
1. 创建图像显示窗口。
2. 创建控制滑块，可以通过这些控制滑块来让你的小车稳定的运行起来，我们创建的控制滑块包括：
> 1. 小车行驶速度（m/s）；
> 2. 转向增益（值越大，小车转弯角度越大，相反，小车转弯角度越小）；
> 3. 回正转向增益（在小车转弯后对转向的一个修正，避免转得过多）；
> 4. 偏移（如果小车行驶的时候手动让它往左偏一点还是往右偏一点）。
3. 创建小车状态信息滑块，包括：
> 1. 经过路径跟踪模型预测出来的x,y的值；
> 2. 经过pid算法计算出小车的转向值；
> 3. 当前小车的速度；
> 4. 激光雷达深度值（用来判断标识牌与小车的距离，单位为：m）。

In [ ]:
image_widget = widgets.Image(format='jpeg', width=300, height=300)

# 控制滑块
speed_gain_slider = ipywidgets.FloatSlider(min=-0.3, max=0.3, step=0.01, value=0.10, description='速度')
steering_gain_slider = ipywidgets.FloatSlider(min=0.0, max=1.0, step=0.01, value=0.50, description='转向增益')
steering_dgain_slider = ipywidgets.FloatSlider(min=0.0, max=1.0, step=0.01, value=0.15, description='回正微调')
steering_bias_slider = ipywidgets.FloatSlider(max=1.0, min=-1.0, step=0.01, value=0.0, description='偏移')
turn_gain_slider = ipywidgets.FloatSlider(min=0, max=1, step=0.01, value=0.23, description='转弯速度')
# 小车状态信息滑块
x_slider = ipywidgets.FloatSlider(min=-1.0, max=1.0, description='预测x值')
y_slider = ipywidgets.FloatSlider(min=0, max=1.0, orientation='vertical', description='预测y值')
steering_slider = ipywidgets.FloatSlider(min=-1.0, max=1.0, description='方向')
speed_slider = ipywidgets.FloatSlider(min=-1.0, max=1.0, description='速度')
depth_slider = ipywidgets.FloatSlider(min=0.0, max=10000.0, description='距离')
label_widget = widgets.IntSlider(value=1, min=0, max=4,description='定义目标')

## 4.数据预处理
因为在训练时我们对训练数据集进行过数据预处理，因此我们在识别的时候，也需要给机器人看到的图片数据做相应的预处理，因此需要回顾一下我们训练标识牌模型和路径跟踪模型时是如何给图片进行预处理的。
需要注意的是，虽然在识别时我们都是用的同一帧机器人看到的画面，但是当经过不同的模型，需要对图片进行不同的预处理工作。

In [ ]:
def preprocess(image,flag):
    img_size = 224
    image = cv2.resize(image,(img_size, img_size),interpolation=cv2.INTER_CUBIC)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = image.transpose((2, 0, 1))
    image = np.ascontiguousarray(image, dtype=np.float32)
    if flag=='signal':
        signal_mean = 255.0 * np.array([0.528, 0.518, 0.483])
        signal_stdev = 255.0 * np.array([0.302, 0.302, 0.311])
        signal_normalize = transforms.Normalize(signal_mean, signal_stdev)
        image = signal_normalize(torch.from_numpy(image)).unsqueeze(0).cuda()
    else:
        steering_mean = 255.0 * np.array([0.44, 0.453, 0.444])
        steering_stdev = 255.0 * np.array([0.249, 0.24, 0.242])
        steering_normalize = transforms.Normalize(steering_mean, steering_stdev)
        image = steering_normalize(torch.from_numpy(image)).unsqueeze(0).cuda().half()
    return image

## 5.检测模型是否能正常使用

我们通过numpy创建与我们将要预测的图片格式一致的形状为（224，224，3）的数组，这里我们创建的全为1的数组将这个数组经过预处理再将数据放入模型中，如果能运行通过说明模型可以正常使用了。

In [ ]:
try:
    img_data = np.ones([224, 224, 3],np.float32)
    road_model(preprocess(img_data, flag='road_following'))
    detect_model(preprocess(img_data, flag='signal'))
    print('模型正确')
except:
    print('请检查模型是否正确')

## 6.标识牌识别
控制小车在遇到对应的标识牌做出相应的动作，再加入深度信息，让小车先判断前方是否有物体，判断距离为0.6米，如果有物体就进行标识牌识别，当小车认为某一种标识牌的概率大于60%的时候，就做出相应的动作。

In [ ]:
classes = ['flower_shop', 'gym', 'background', 'park', 'store', 'turn','zoo' ]

global pred_index
pred_index=''

global see_stop_flag
see_stop_flag = False

def signal_detection(img_data, depth):
    global pred_index
    selective_classes = ['flower_shop','gym','park', 'store', 'zoo']
    predection = detect_model(preprocess(img_data, flag='signal'))
    output = F.softmax(predection, dim=1)
    prob, pred_index = torch.max(output, 1)
    label = classes[pred_index]
    select_index = int(label_widget.value)
    target =  selective_classes[select_index]

    # 只在目标建筑前停止，在其他建筑前进行掉头，并且忽略背景类别。
    cv2.putText(img_data, 'target is '+ target, (20, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,0,255), 1, cv2.LINE_AA)

    if prob.item() > 0.6:
        if label == 'turn':
            cv2.putText(img_data, 'turn signal', (20, 40), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,255,0), 1, cv2.LINE_AA)
            rbt.base.turnleft(0.3,10)
        if depth < 0.5:
            if label == target:
                cv2.putText(img_data, 'find target ', (20, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,255,0), 1, cv2.LINE_AA)
                see_stop_flag = True
            
            elif label!=target and label in selective_classes:
                cv2.putText(img_data, 'prediction is '+label, (20, 40), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255,255,255), 1, cv2.LINE_AA)
                rbt.base.turnleft(0.3,10)
                see_stop_flag = False
                
    return see_stop_flag

## 7.定义路径跟踪
机器人将通过路径追踪模型推断出当前图片的x,y值，通过x,y值来计算角度值，再通过调节之前创建的控制滑块对小车的行驶进行微调就可以计算出小车最终的转向值，计算的细节在“pid”变量里边。

In [ ]:
def road_following(img_data, angle, angle_last):    
    xy = road_model(preprocess(img_data, flag='steering')).detach().half().cpu().numpy().flatten()
    x = xy[0]
    ys = (0.5 - xy[1]) / 2.0
    angle = -np.arctan2(x, ys)
    
    pid = angle * steering_gain_slider.value - (angle + angle_last)*steering_dgain_slider.value
    # 手动改变偏移值
    steering = pid - steering_bias_slider.value
    
    speed = speed_gain_slider.value  * (1+turn_gain_slider.value-xy[1])
    steering = steering * xy[1]
    x_slider.value = x
    y_slider.value = ys
    speed_slider.value = speed
    steering_slider.value = -steering
    rbt.base.move(speed, 0, steering)
    angle_last = angle

## 8.运行路径跟踪和标识牌识别
多尝试用控制滑块来调节小车的运动，让小车稳定下来，如果机器人还是做得不够好，很可能是因为你收集的数据不够或者说收集数据时没按照要求进行收集，多多尝试，找出问题的原因。

### 8.1.创建深度获取函数

In [ ]:
# 创建滑块，将深度信息进行可视化
depth_slider = ipywidgets.FloatSlider(min=0.0, max=10000.0, description='深度值')

# 创建深度图像显示窗口
depth_image_widget = widgets.Image(format='jpeg')

# 初始化深度值，角度值和上一次识别的角度值
global depth 
depth = 0.0

def on_new_depth(evt):
    global depth
    depth = evt.dict['data'].get_distance(310, 90)
    if depth ==0:
        depth = evt.dict['data'].get_distance(330, 85)
        
    depth_slider.value = depth
    
    depth_frame = np.asanyarray(evt.dict['data'].get_data())
    depth_colormap = cv2.applyColorMap(cv2.convertScaleAbs(depth_frame, alpha=0.03), cv2.COLORMAP_JET)
    depth_colormap = cv2.resize(depth_colormap, (320,240))
    depth_image_widget.value = bgr8_to_jpeg(depth_colormap)

### 8.2.创建图像获取函数

In [ ]:

angle = 0.0
angle_last = 0.0
global threading_stop
threading_stop=False

def _prediction():
    global threading_stop
    while threading_stop==False:
        global see_stop_flag
        time.sleep(0.001)
        img_data = rbt.camera.read()
        if img_data is not None:
            global depth
            # 标识牌识别
            if rbt_name=='dachbot':
                see_stop_flag = signal_detection(img_data,depth)
            # 路径跟踪
            if see_stop_flag==False:
                global angle, angle_last
                road_following(img_data, angle, angle_last)
            else: 
                rbt.base.stop()
            img_data = cv2.resize(img_data, (320,240))
            image_widget.value=bgr8_to_jpeg(img_data)
        
    print('模型预测线程已关闭！')


## 9.使用线程说出预测结果

In [ ]:
import threading
voice = ['花店到了','体育馆到了', '','公园到了', '便利店到了', '开始掉头', '动物园到了']

def interaction():
    while threading_stop==False:
        global pred_index
        if pred_index!='':
            rbt.speech.play_text(voice[pred_index], True)
            pred_index=''
        time.sleep(1)
    print('语音线程已关闭！')

## 10.开始运行
连接小车，打开摄像头。
* 可以选择目标类别，到达目标类别就停下，其它类别就掉头，类别分别有：['花店到了','体育馆到了','公园到了', '便利店到了', '动物园到了']
* 对应的索引位置分别为[0,1,2,3,4],我们通过选择索就代表选择的对应类别。

In [ ]:

rbt = Robot()
rbt.connect()

rbt_name = rbt.name

if rbt_name=='dachbot':
    rbt.base.set_transform(False)
    rbt.camera.start(enable_depth_stream=True)
    rbt.event_manager.add_event_listener(event.EventTypes.NEW_CAMERA_DEPTH,on_new_depth)
    process1 = threading.Thread(target=interaction,)
    process1.start()
    rbt.speech.start()
    display(ipywidgets.HBox([image_widget,depth_image_widget, x_slider,y_slider]))
    display(ipywidgets.VBox([steering_slider, speed_slider,depth_slider]))
elif rbt_name=='dbot':
    rbt.camera.start()
    display(ipywidgets.HBox([image_widget, x_slider,y_slider]))
    display(ipywidgets.VBox([steering_slider, speed_slider]))
    
process2 = threading.Thread(target=_prediction,)
process2.start()


print('控制滑块-操纵小车')
display(ipywidgets.VBox([label_widget, speed_gain_slider, steering_gain_slider,
                        steering_dgain_slider, turn_gain_slider, steering_bias_slider]))

rbt.base.set_ptz(-22)

## 11.断开与小车的连接

In [ ]:
# if rbt_name=='dachbot':
#     rbt.base.set_transform(False)
# threading_stop=True
# rbt.disconnect()